In [1]:
import numpy as np
import pandas as pd
import csv
from dask.distributed import Client, progress
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

In [2]:
df = pd.read_csv('data/train2.csv', dtype={"district_id": "Int64", "city_id": "Int64", "id": "Int64", "is_business": "Int64", "region_id": "Int64", "price": "Int64", "created_at_first": "str",
"params": "str"}, engine="python", encoding="utf-8")

In [3]:
districts = pd.read_csv("data/districts.csv", dtype={'id': 'Int64'})

In [4]:
cities = pd.read_csv("data/cities.csv", dtype={'id': 'Int64'})

In [5]:
data = pd.merge(df, cities, left_on="city_id", right_on="id")

In [6]:
data = data.drop("id_y", axis=1)

In [7]:
districts = districts.rename(columns={"id": "district_id"})

In [8]:
data2 = pd.merge(data, districts, on="district_id", how="left")

In [10]:
data = data2.drop(columns = {"Unnamed: 0", "id_x", "created_at_first", "region_id", "lon_x", "lat_x"})

In [11]:
data = data.rename(columns = {"name_pl_x": "city_name", "name_pl_y": "disctrict_name"})

In [12]:
data.head()

,category,is_business,district_id,city_id,params,price,city_name,disctrict_name,lon_y,lat_y
0,Mieszkanie na sprzedaż,1,12867,210,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556,Olsztyn,Dajtki,20.42342,53.76516
1,Mieszkanie na sprzedaż,1,280,210,price[currency]<=>PLN<br>m<=>49.6<br>rooms_num...,222000,Olsztyn,Kormoran,20.49830,53.76785
2,Mieszkanie na sprzedaż,1,590,210,price[currency]<=>PLN<br>m<=>39.41<br>rooms_nu...,254195,Olsztyn,Nagórki,20.49327,53.75620
3,Dom na sprzedaż,1,2568,210,price[currency]<=>PLN<br>terrain_area<=>703<br...,690000,Olsztyn,Redykajny,20.43742,53.81064
4,Mieszkanie na sprzedaż,1,6168,210,price[currency]<=>PLN<br>m<=>36.53<br>rooms_nu...,209000,Olsztyn,Osiedle Mazurskie,20.51410,53.76086


In [15]:
def parse_values(row):
    values = row.split('<br>')
    parsed_values = {}
    for value in values:
        if value.strip():
            col_name, val = value.split('<=>')
            parsed_values[col_name.strip()] = val.strip()
    return parsed_values

# Apply the function to each row and convert the resulting list of dictionaries into a DataFrame
data_test_algorithm = data.head()
pd.set_option('display.max_columns', None)
parsed_data = data_test_algorithm['params'].apply(parse_values).apply(pd.Series)
parsed_data.head()

,price[currency],m,rooms_num,market,building_type,floor_no,building_floors_num,building_material,windows_type,heating,build_year,construction_status,rent,rent[currency],building_ownership,free_from,media_types,security_types,equipment_types,extras_types,terrain_area,floors_num,garret_type,roof_type,roofing,location,recreational,fence_types,heating_types,access_types,vicinity_types,is_bungalow
0,PLN,72.14,4,secondary,,floor_1,1,,plastic,,2018,,,PLN,,,,,,balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLN,49.6,2,secondary,block,floor_2,2,,plastic,urban,1958,to_renovation,350,PLN,full_ownership,,,,,balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PLN,39.41,2,primary,block,,,,,,,,,PLN,,,0,0,0,garden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PLN,250,5,secondary,semi_detached,NaN,NaN,,plastic,NaN,,ready_to_use,,PLN,NaN,,electricity<->gas<->sewage,,NaN,garage,703,,,,tile,,,,,asphalt,,0
4,PLN,36.53,1,secondary,block,floor_1,4,,plastic,urban,2017,ready_to_use,250,PLN,full_ownership,,,,,basement<->lift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
